In [ ]:
import pandas as pd
import warnings
import re
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np

# warning 없애기
warnings.filterwarnings("ignore")

# 글씨체 깨짐 방지
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


df = pd.read_csv('C:/Users/wndus/OneDrive/바탕 화면/관광지추천/2차_전처리.csv', encoding = 'utf-8')
df

# 1. 1인 1일 지출경비

In [ ]:
target_col = '문10. 1인 1일 지출경비(영향치 결측, 61일 이상 결측, 국제교통비 제외)'

plt.figure(figsize=(8,5))
plt.hist(df[target_col], bins=40, edgecolor='black', alpha=0.7)
plt.xlabel('1인 1일 지출경비')
plt.ylabel('Count')
plt.title('지출경비 분포 (Histogram)')
plt.show()


In [ ]:
quantiles = np.arange(0, 1.01, 0.05)   # 0%, 5%, 10%, ..., 100%
q_table = df[target_col].quantile(quantiles)

q_df = q_table.reset_index()
q_df.columns = ['Quantile', 'Value']
q_df

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(q_df['Quantile'], q_df['Value'], marker='o')
plt.xlabel('Quantile (%)')
plt.ylabel('지출경비 값')
# plt.xlim(0, 0.95) 
# plt.ylim(0, 600)
plt.title('지출경비 분위수 분포 (5% 단위)')
plt.grid(True)
plt.show()

In [ ]:
# q_df 는 다음 형태로 이미 존재한다:
# Quantile | Value

q = q_df['Quantile'].values
v = q_df['Value'].values

# 인접 분위수 간 차이 계산
diff = np.diff(v)  # Value 차이
step = np.diff(q)  # Quantile 차이 (여기서는 전부 5씩)

# 기울기(증가량) = diff / step 이지만 step은 일정하므로 diff 자체로 비교 가능
slope = diff / step

# 구간 테이블 생성
slope_df = pd.DataFrame({
    'q_start': q[:-1],
    'q_end': q[1:],
    'v_start': v[:-1],
    'v_end': v[1:],
    'diff': diff,
    'slope': slope
})

# 기울기 큰 순으로 7개 구간 추출
top7 = slope_df.reindex(slope_df['slope'].abs().sort_values(ascending=False).index).head(7)

top7

In [ ]:
slope_df.reindex(slope_df['slope'].abs().sort_values(ascending=False).index)

# 90 ~ 100 % 
# 65 ~ 90 %
# 55 ~ 65 %
# 15 ~ 55 % 
# 5 ~ 15 %
# 0 ~ 5 %

In [ ]:
### 구간화 ###

col = '문10. 1인 1일 지출경비(영향치 결측, 61일 이상 결측, 국제교통비 제외)'

# 분위수 기준값
p20 = 106.08
p50 = 177.765
p80 = 298.14
p90 = 400
p95 = 540.9375

def segment(x):
    if x < p20:
        return 1
    elif x < p50:
        return 2
    elif x < p80:
        return 3
    elif x < p90:
        return 4
    elif x < p95:
        return 5
    else:
        return 6

df[col] = df[col].apply(segment)


df['문10. 1인 1일 지출경비(영향치 결측, 61일 이상 결측, 국제교통비 제외)'].value_counts()

# 2. 총 체제기간

In [ ]:
target_col2 = '문9-3. 총 체재기간_61일 이상 결측'

plt.figure(figsize=(8,5))
plt.hist(df[target_col2], bins=40, edgecolor='black', alpha=0.7)
plt.xlabel('총 체재기간 (일)')
plt.ylabel('Count')
plt.title('총 체재기간 분포 (Histogram)')
plt.show()


In [ ]:
quantiles = np.arange(0, 1.01, 0.05)   # 0%, 5%, 10%, ..., 100%
q_table2 = df[target_col2].quantile(quantiles)

q_df2 = q_table2.reset_index()
q_df2.columns = ['Quantile', 'Value']
q_df2


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(q_df2['Quantile'], q_df2['Value'], marker='o')
plt.xlabel('Quantile (%)')
plt.ylabel('체재기간 (일)')
plt.title('총 체재기간 분위수 분포 (5% 단위)')
plt.grid(True)
plt.show()

In [ ]:
q = q_df2['Quantile'].values
v = q_df2['Value'].values

# 인접 분위수 간 차이 계산
diff = np.diff(v)      # Value 차이
step = np.diff(q)      # Quantile 차이 (여기서는 전부 5씩 일정)

# 기울기(증가량) = diff / step (step 일정이라 diff로 비교해도 동일)
slope = diff / step

# 구간 테이블 생성
slope_df2 = pd.DataFrame({
    'q_start': q[:-1],
    'q_end': q[1:],
    'v_start': v[:-1],
    'v_end': v[1:],
    'diff': diff,
    'slope': slope
})

# 기울기(절댓값) 큰 순으로 상위 7개 구간 추출
top7_stay = slope_df2.reindex(slope_df2['slope'].abs().sort_values(ascending=False).index).head(7)

top7_stay


In [ ]:
### 구간화 ###

col = '문9-3. 총 체재기간_61일 이상 결측'

segments = []

for x in df[col]:
    if 1 <= x <= 3:
        segments.append(1)
    elif 4 <= x <= 5:
        segments.append(2)
    elif 6 <= x <= 7:
        segments.append(3)
    elif 8 <= x <= 11:
        segments.append(4)
    elif 12 <= x <= 17:
        segments.append(5)
    elif x >= 18:
        segments.append(6)
    else:
        segments.append(np.nan)  # 혹시 0이나 이상값 있을 때 대비
        

df['문9-3. 총 체재기간_61일 이상 결측'] = segments
df['문9-3. 총 체재기간_61일 이상 결측'].value_counts()

# 3. 항목별 지출경비

In [ ]:
import pandas as pd

df1 = pd.read_csv('C:/Users/wndus/OneDrive/바탕 화면/관광지추천/만족도조사.csv', encoding = 'utf-8')
df1 = df1[df1['문1. 주요 방한 목적'] == 1]
df1

In [ ]:
# 1. df1에서 지출 컬럼 자동 선택
expense_cols = [col for col in df1.columns if col.startswith('문10-1. 항목별 지출경비(상하위 1% 대체)_')]

# 2. ID + 선택된 컬럼만 유지
df1_use = df1[['아이디'] + expense_cols]

# 3. merge
df = df.merge(df1_use, on='아이디', how='left')
df

In [ ]:
expense_groups = {
    '식비': [
        '문10-1. 항목별 지출경비(상하위 1% 대체)_음식점 지출비',
        '문10-1. 항목별 지출경비(상하위 1% 대체)_식음료/식재료 구매 비용'
    ],
    
    '여행_가이드': [
        '문10-1. 항목별 지출경비(상하위 1% 대체)_여행사에 지불한 경비',
        '문10-1. 항목별 지출경비(상하위 1% 대체)_가이드 비용/가이드 팁'
    ],
    
    '숙박': [
        '문10-1. 항목별 지출경비(상하위 1% 대체)_숙박비'
    ],
    
    '교통': [
        '문10-1. 항목별 지출경비(상하위 1% 대체)_한국에서의 교통비_항공비(국적기)',
        '문10-1. 항목별 지출경비(상하위 1% 대체)_한국에서의 교통비_항공비(국적기 외)',
        '문10-1. 항목별 지출경비(상하위 1% 대체)_한국에서의 교통비_수상 여객운송',
        '문10-1. 항목별 지출경비(상하위 1% 대체)_한국에서의 교통비_철도 여객운송',
        '문10-1. 항목별 지출경비(상하위 1% 대체)_한국에서의 교통비_도로 여객운송',
        '문10-1. 항목별 지출경비(상하위 1% 대체)_운송장비 대여서비스',
        '문10-1. 항목별 지출경비(상하위 1% 대체)_유류비'
    ],
    
    '체험_문화': [
        '문10-1. 항목별 지출경비(상하위 1% 대체)_문화서비스',
        '문10-1. 항목별 지출경비(상하위 1% 대체)_오락 및 운동관련 서비스'
    ],
    
    '쇼핑': [
        '문10-1. 항목별 지출경비(상하위 1% 대체)_쇼핑비'
    ],
    
    '기타': [
        '문10-1. 항목별 지출경비(상하위 1% 대체)_데이터 통신비',
        '문10-1. 항목별 지출경비(상하위 1% 대체)_기타 비용'
    ],
    
    '치료': [
        '문10-1. 항목별 지출경비(상하위 1% 대체)_치료 및 건강검진비'
    ],
    
    '뷰티_미용': [
        '문10-1. 항목별 지출경비(상하위 1% 대체)_미용 서비스비'
    ]
}


In [ ]:
# 그룹별 지출 합 변수 생성
for group, cols in expense_groups.items():
    df[f'총_{group}_지출경비'] = df[cols].sum(axis=1)
    

In [ ]:
# 총 국내 지출경비 생성
df['총_국내_지출경비'] = df[[f'총_{g}_지출경비' for g in expense_groups]].sum(axis=1)
df['총_국내_지출경비']

In [ ]:
# 비율 변수 생성
for group in expense_groups.keys():
    df[f'{group}_비율'] = (df[f'총_{group}_지출경비'] / df['총_국내_지출경비']).round(3)


In [ ]:
drop_cols = [col for col in df.columns 
             if col.startswith('문10-1. 항목별 지출경비(상하위 1% 대체)_')]

df = df.drop(columns=drop_cols)
df.columns

In [ ]:
df = df.drop(columns= ['Unnamed: 0.1', 'Unnamed: 0', '문1. 주요 방한 목적'])
df

## 매핑

In [ ]:
mapping = {
    1: 1,
    2: 2,
    3: 3,
    4: 4, 10: 4,
    5: 5, 6: 5,
    7: 6,
    8: 7, 9: 7,
    11: 8
}

df['문1-1. 한국여행 관심 계기_1순위'] = df['문1-1. 한국여행 관심 계기_1순위'].map(mapping)
df['문1-1. 한국여행 관심 계기_1순위'].value_counts()

In [ ]:
mapping = {
    1: 1,
    2: 2,
    3: 3,
    4: 4, 5: 4,
    6: 5, 8: 5,
    7: 6,
    14: 7, 15: 7,
    9: 8,
    10:9, 11:9,
    12:10, 13:10,
    16:11
}

df['문3-1. 고려한 관광활동_1순위'] = df['문3-1. 고려한 관광활동_1순위'].map(mapping)
df['문3-1. 고려한 관광활동_1순위'].value_counts()

In [ ]:
mapping = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6, 8:6,
    7: 7, 
    9:8, 10:8, 11:8,
    12:9
}

df['문3-2. 고려한 관광인프라_1순위'] = df['문3-2. 고려한 관광인프라_1순위'].map(mapping)
df['문3-2. 고려한 관광인프라_1순위'].value_counts()

In [ ]:
mapping = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6, 8:6,
    7: 7, 
    9:8, 10:8, 11:8,
    12:9
}

df['문3-2. 고려한 관광인프라_1순위'] = df['문3-2. 고려한 관광인프라_1순위'].map(mapping)
df['문3-2. 고려한 관광인프라_1순위'].value_counts()

In [ ]:
df.to_csv('C:/Users/wndus/OneDrive/바탕 화면/관광지추천/4차_전처리.csv')